## Import code from gitHub

In [0]:
_git_branch = 'charters-subjects-2'

#Document parser, refer https://github.com/nemoware/document-parser/releases
lib_version = '1.1.18'
document_parser_lib_version=lib_version

In [0]:
from google.colab import files


In [0]:
import json
import subprocess
import sys
import time

import numpy as np
from IPython.core.display import display, HTML
from google.colab import files

!pip install overrides

Ничто = None


def exec(x):
  r = subprocess.check_output(x, shell=True)
  r = r.decode('unicode-escape').encode('latin1').decode('utf8')
  print(r)


print(f"fetching code from GitHub.....{_git_branch}")
try:
  exec('rm -r nlp_tools')
except:
  pass
exec(f'git clone --single-branch --branch {_git_branch} https://github.com/nemoware/analyser.git nlp_tools')

print('🦊 GIT revision:')
exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

sys.path.insert(0, 'nlp_tools')

print('❤️importing Code from GitHub ... DONE')

#----- other deps
!pip install pyjarowinkler
import nltk
nltk.download('punkt')

In [0]:
#----
import matplotlib as mpl
from analyser.documents import TextMap
from analyser.legal_docs import DocumentJson
from colab_support.renderer import HtmlRenderer

 

class DemoRenderer(HtmlRenderer):
  def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    html = self.to_color_text(tokens, weights, colormap, print_debug, _range, separator=separator)
    display(HTML(html))

  def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range, separator=separator)

   
renderer_ = DemoRenderer()

def print_json_summary(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
  print(f'read file {cd.filename}')

  for tag in cd.tags:
    span = tag.span
    _map = cd.tokenization_maps[tag.span_map]
    print(tag)

### Load pre-embedded test doc

In [0]:
#@title Pre-embedded or Upload?
upload_files = True #@param {type:"boolean"}


### Init document-parser lib

In [0]:
lib_version = document_parser_lib_version
import os
if not os.path.isfile(f'document-parser-{lib_version}-distribution.zip'):
  !wget https://github.com/nemoware/document-parser/releases/download/$lib_version/document-parser-$lib_version-distribution.zip
if not os.path.isdir(f'document-parser-{lib_version}'):
  !unzip document-parser-$lib_version-distribution.zip

 
os.environ ['documentparser']=f'/content/document-parser-{lib_version}'
from integration.word_document_parser import WordDocParser, join_paragraphs
wp = WordDocParser()

### imports

In [0]:
import os
import pickle
import unittest

import numpy as np

from analyser.contract_parser import ContractAnlysingContext, ContractDocument
#from analyser.contract_parser import find_value_sign_currency_attention
from analyser.contract_patterns import ContractPatternFactory
 
 
# from headers_detector import doc_features, load_model, make_headline_attention_vector
from analyser.hyperparams import HyperParameters
from analyser.contract_parser import ContractAnlysingContext
from analyser.parsing import AuditContext

from analyser.legal_docs import LegalDocument, ContractValue
from analyser.ml_tools import *
from analyser.patterns import *
from analyser.text_tools import *

from tf_support.embedder_elmo import ElmoEmbedder
 

## 💅 Init Embedder(s)

In [0]:
elmo_embedder = ElmoEmbedder()  

## Init parser

In [0]:
ctx = ContractAnlysingContext(elmo_embedder)

# Read doc

In [0]:
#@title ✅Read uploaded file

filename = '/content/8.\u0434\u043E\u0433\u043E\u0432\u043E\u0440.doc' #@param {type:"string"} 
trim_doc =  False #@param {type:"boolean"} 
 
results = wp.read_doc(filename)
for doc in results['documents'][:1]:  # XXX
  if 'CONTRACT' == doc['documentType']:    
    doc = join_paragraphs(doc, 'no_id')

# print(doc.original_text)

In [0]:
for p in doc.paragraphs:
  print ('☢️', p.header.value.strip())

In [0]:
## ACHTUNG!
## TRIM doc, to make debug faster
if trim_doc:
  doc = doc[0:1500]
  doc.paragraphs=None

print(doc.text)

# Parse Contract

## phase 0

In [0]:

actx = AuditContext()
ctx.find_org_date_number(doc, actx)

### render phase 0 results

In [0]:
renderer_.render_color_text(doc.tokens[0:300],  doc.get_tags_attention()[0:300] )

## Phase 1

In [0]:
ctx.find_attributes(doc, actx)

### render phase 1 results

In [0]:
renderer_.render_color_text(doc.tokens,  doc.get_tags_attention()  )

In [0]:
for t in doc.get_tags():
  print('\n',t, '-'*80)
  print(doc.substr(t))

### structure

In [0]:
# ctx.sections_finder.find_sections(doc, ctx.pattern_factory, ctx.pattern_factory.headlines,
#                                        headline_patterns_prefix='headline.')


### subject

IN doc head

In [0]:
section = doc[0:1500]
denominator = 0.7
result = ctx.find_contract_subject_regions(section, denominator)
print(result)

print( doc.substr(result))

in Subject section

In [0]:



# subject_section = doc.sections['subj'].body

subject_section = doc #or select a section
tag = ctx.find_contract_subject_regions(subject_section, 1)
print(tag)
print(subject_section.substr(tag))

#### subject details

In [0]:
subject_headline_attention = subject_section.distances_per_pattern_dict ['subject_headline_attention'] 
subject_headline_attention_max = max(subject_headline_attention)
print('subject_headline_attention_max',subject_headline_attention_max)
section=subject_section
# all_subjects_headlines_vectors = filter_values_by_key_prefix(subject_section.distances_per_pattern_dict, 'headline.subj')
# subject_headline_attention: FixedVector = max_exclusive_pattern(all_subjects_headlines_vectors)
# subject_headline_attention = best_above(subject_headline_attention, 0.5)
# subject_headline_attention = momentum_t(subject_headline_attention, half_decay=20)


# section=subject_section
# all_subjects_headlines_vectors = filter_values_by_key_prefix(section.distances_per_pattern_dict, 'headline.subj')

# subject_headline_attention: FixedVector = max_exclusive_pattern(all_subjects_headlines_vectors)
# subject_headline_attention = best_above(subject_headline_attention, 0.5)
# subject_headline_attention = momentum_t(subject_headline_attention, half_decay=80)


In [0]:

import matplotlib as mpl
import matplotlib.pyplot as plt
 
plt.rcParams["figure.figsize"] = (17,4)
fig = plt.figure()  
ax = plt.axes(title=f'subject_headline_attention')

all_subjects_headlines_vectors = filter_values_by_key_prefix(section.distances_per_pattern_dict, 'headline.subj')
subject_headline_attention: FixedVector = max_exclusive_pattern(all_subjects_headlines_vectors)
ax.plot(subject_headline_attention)

subject_headline_attention = best_above(subject_headline_attention, 0.2)
ax.plot(subject_headline_attention)

subject_headline_attention = momentum_t(subject_headline_attention, half_decay=120)
ax.plot(subject_headline_attention)


print(max(subject_headline_attention))

##### ContractSubjects

In [0]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from analyser.contract_parser import contract_subjects, _find_most_relevant_paragraph
plt.rcParams["figure.figsize"] = (17,4)
 

subject_headline_attention_max = max(subject_headline_attention)
for subject_kind in contract_subjects:
# subject_kind = ContractSubject.RealEstate
  print("="*20)
  print(subject_kind)
  subject_attention_vector: FixedVector = ctx.make_subject_attention_vector_3(subject_section, subject_kind, None)
  if subject_headline_attention_max > 0.5:
    subject_attention_vector *= subject_headline_attention
  print(len(subject_attention_vector))
  paragraph_span, confidence, paragraph_attention_vector = _find_most_relevant_paragraph(subject_section,
                                                                                             subject_attention_vector,
                                                                                             min_len=20,
                                                                                             return_delimiters=False)
  
  # confidence_region = subject_attention_vector# subject_attention_vector[paragraph_span[0]:paragraph_span[1]]
  # confidence_alt = estimate_confidence_by_non_zeros(subject_attention_vector)
  print(confidence,  len(paragraph_attention_vector),'\n', subject_section.tokens_map.text_range(paragraph_span))

  renderer_.render_color_text(subject_section.tokens,  subject_attention_vector, _range=(0,1))

  fig = plt.figure()  
  ax = plt.axes(title=f'{subject_kind}')
  
  
  # ax.plot(smooth(subject_attention_vector, 30),color=(0,0,1,0.5) )   
  
  _mark=[max(subject_attention_vector)]*len(subject_attention_vector)
  _mark[0:paragraph_span[0]]*=0
  _mark[paragraph_span[1]:]*=0
  ax.plot(_mark, color=(0,0,0,0.5) )   

  _mark=[confidence]*len(subject_attention_vector)
  _mark[0:paragraph_span[0]]*=0
  _mark[paragraph_span[1]:]*=0
  ax.plot(_mark, color=(0,0,0,0.5) )   

  
  ax.plot( subject_headline_attention , color=(0,1,0,0.5) )   

  ax.plot(subject_attention_vector, color=(0,0,0,0.5) )   
  ax.plot(paragraph_attention_vector, color=(1,0,0,0.5) )   
  
  #thresholds
  
  ax.plot([max(subject_headline_attention)]*len(subject_headline_attention), color=(0,1,0,0.5) )   
  ax.plot([max(paragraph_attention_vector)]*len(paragraph_attention_vector), color=(1,0,0,0.5) )   


### Paragraphs mapping to topics AKA sections

print all headers

In [0]:
for p in doc.paragraphs:
  print ('☢️', p.header.value.strip())

In [0]:
factory = ctx.pattern_factory


head_subject_patterns_prefix = 'hds_' 

# XXX: this has been moved to contraact_parser.py
contract_headlines_patterns={
  'КУПЛИ-ПРОДАЖИ НЕДВИЖИМОГО ИМУЩЕСТВА':ContractSubject.RealEstate,
  'купли-продажи':ContractSubject.Deal,  
  'займа':ContractSubject.Loans,
  'оказания консультационных услуг':ContractSubject.Consulting,
  'на оказание услуг':ContractSubject.Service,
  'залога':ContractSubject.PledgeEncumbrance,
  'о безвозмездной помощи ( Пожертвование )':ContractSubject.Charity,
  'безвозмездного пользования нежилым помещением':ContractSubject.Charity,
  'генерального подряда':ContractSubject.GeneralContract,
  'аренды недвижимого имущества':ContractSubject.Renting
}

 

# XXX: this has been moved to contraact_parser.py
def _build_head_subject_patterns(self):

  for txt in contract_headlines_patterns:
    cnt = len(self.patterns)    
    kind = contract_headlines_patterns[txt]
    self.create_pattern(f'{head_subject_patterns_prefix}{kind}.{cnt}',
                        ('Договор', txt.lower().strip(), ''))
    
   

_build_head_subject_patterns(factory)
factory.embedd()

In [0]:




# XXX: this has been moved to contraact_parser.py
def match_headline_to_subject(section: LegalDocument, subject_kind: ContractSubject) -> FixedVector:

  pattern_prefix = f'{head_subject_patterns_prefix}{subject_kind}'
  _vectors = list(  filter_values_by_key_prefix(section.distances_per_pattern_dict, pattern_prefix) )

  if not _vectors:
    warnings.warn(f'no patterns for {subject_kind}')
    return np.zeros(len(section.tokens_map))
    
  vectors = [ best_above(v, 0.4) for v in _vectors ]

  x = max_exclusive_pattern(vectors)
  x = relu(x, 0.6)
  
  return x

# XXX: this has been moved to contraact_parser.py
def find_headline_subject_match(doc: LegalDocument, factory: AbstractPatternFactory) -> (ContractSubject, float):
  headers = [doc.subdoc_slice(p.header.as_slice()) for p in doc.paragraphs]

  max_confidence = 0
  best_subj = None
  for header_index, header in enumerate(
          headers[0:3]):  # take only 3 fist headlines; normally contract type is known by the 1st one.

    if header.text and header.text.strip():

      vvs = header.calculate_distances_per_pattern(factory, pattern_prefix=head_subject_patterns_prefix, merge=False)
      

      for subject_kind in contract_headlines_patterns.values():  # like ContractSubject.RealEstate ..
        subject_attention_vector: FixedVector = match_headline_to_subject(header, subject_kind)
        _confidence = estimate_confidence_by_mean_top_non_zeros(subject_attention_vector)
        if _confidence > max_confidence:
          max_confidence = _confidence
          best_subj = subject_kind

        # print (subject_kind, _confidence)

  return best_subj, max_confidence


best_subj, max_confidence = find_headline_subject_match(doc, factory)
print ('-'*20)
print (best_subj, max_confidence)

In [0]:
#Find sections
sections = ctx.sections_finder.find_sections(doc, 
                                  ctx.pattern_factory, 
                                  ctx.pattern_factory.headlines,
                                  headline_patterns_prefix='headline.')
for s in sections:
  print(f'☢️{sections[s].confidence:.4f} \t {s}\t\t  {sections[s].header.strip()} \t')
#

In [0]:

from analyser.ml_tools import *

confidence_threshold=0.7
sections_filtered = {}
for section_type in factory.headlines:
  print(section_type, '-'*100)
  pattern_prefix = f'headline.{section_type}'

  #find best header for each section
  headers = [contract.subdoc_slice(p.header.as_slice()) for p in contract.paragraphs]
  best_header = headers[0]

  _max_confidence = 0
  _max_header_i = -1
  for header_index in range(len(headers)):
    header = headers[header_index]
    vvs = header.calculate_distances_per_pattern(factory, pattern_prefix=pattern_prefix, merge=False)
    vv = max_exclusive_pattern( list( vvs.values()) ) #bayes-aggregated vectors
    _confidence = max(vv)
    if _confidence > _max_confidence:
      _max_confidence = _confidence
      _max_header_i = header_index
      best_header = header
    # print( vvs.keys())

    # renderer_.render_color_text(header.tokens,vv , _range=(0,1))
  print (_max_header_i, section_type, _max_confidence, best_header.text)  

  if _max_confidence > confidence_threshold:
    put_if_better(sections_filtered, _max_header_i, (_max_confidence, section_type, best_header.text), lambda a, b: a[1] < b[1])

for k in sections_filtered.values():
  print (k)

In [0]:
# headers = [contract.subdoc_slice(p.header.as_slice()) for p in contract.paragraphs]
# for h in headers:
#   print (h.text)

## Subject

### make_subject_attention_vector_3
найдена секция "предмет договора" (subj)

In [0]:
contract.sections['subj'].header

In [0]:
from ml_tools import SemanticTag, FixedVector, filter_values_by_key_prefix, rectifyed_sum
from structures import ContractSubject

section = contract.sections['subj'].body
 
section.calculate_distances_per_pattern(ctx.pattern_factory, merge=True, pattern_prefix='x_ContractSubject')

all_subjects_vectors = filter_values_by_key_prefix(section.distances_per_pattern_dict, 'x_ContractSubject')
all_subjects_mean: FixedVector = rectifyed_sum(all_subjects_vectors)

print(ContractSubject.Deal)
subject_kind =  ContractSubject.Deal
subject_attention_vector: FixedVector = ctx.make_subject_attention_vector_3(section, subject_kind)

renderer_.render_color_text(section.tokens, subject_attention_vector, _range=(0,1))


print(ContractSubject.Charity)
subject_kind =  ContractSubject.Charity
subject_attention_vector: FixedVector = ctx.make_subject_attention_vector_3(section, subject_kind)

renderer_.render_color_text(section.tokens, subject_attention_vector, _range=(0,1))

In [0]:
print(section.distances_per_pattern_dict.keys())
renderer_.render_color_text(section.tokens, section.distances_per_pattern_dict['$at_x_ContractSubject.Deal'], _range=(0,1))

### make_subject_attention_vector_3 (1500)
первые 1500 слов, когда секция "предмет договора" не найдена

In [0]:
from ml_tools import SemanticTag, FixedVector, filter_values_by_key_prefix, rectifyed_sum
from structures import ContractSubject

#Finding subject in first 1500 words
section = contract.subdoc_slice(slice(0, 1500))

section.calculate_distances_per_pattern(ctx.pattern_factory, merge=True, pattern_prefix='x_ContractSubject')

all_subjects_vectors = filter_values_by_key_prefix(section.distances_per_pattern_dict, 'x_ContractSubject')
all_subjects_mean: FixedVector = rectifyed_sum(all_subjects_vectors)
renderer_.render_color_text(section.tokens, all_subjects_mean)

#### Higlight charity

In [0]:
# from ml_tools import filter_values_by_key_prefix, FixedVector, max_exclusive_pattern, relu
# from patterns import AV_PREFIX, AV_SOFT
# from structures import ContractSubject


# def _subj_attention_names(subj: ContractSubject):
#   a = f'x_{subj}'
#   b = AV_PREFIX + f'x_{subj}'
#   c = AV_SOFT + a
#   return a, b, c

 

subject_kind =  ContractSubject.Charity
subject_attention_vector: FixedVector = ctx.make_subject_attention_vector_3( section, subject_kind, all_subjects_mean)
renderer_.render_color_text(section.tokens, subject_attention_vector)


### _find_most_relevant_paragraph

In [0]:
from legal_docs import LegalDocument
from contract_parser import _find_most_relevant_paragraph
from ml_tools import  *
import numpy as np

 
span, confidence, paragraph_attention_vector = _find_most_relevant_paragraph(section,  subject_attention_vector, min_len=20)

renderer_.render_color_text(section.tokens, section.distances_per_pattern_dict['headline.subj.1'], _range=(0,1))

print(span, confidence)
print (section.tokens_map.text_range(span))

In [0]:
_ =section.calculate_distances_per_pattern(ctx.pattern_factory, merge=True, pattern_prefix='headline.subj')

#### Higlight deal

In [0]:

subject_kind =  ContractSubject.Deal
subject_attention_vector: FixedVector = ctx.make_subject_attention_vector_3(section, subject_kind,
                                                                                   all_subjects_mean)
renderer_.render_color_text(section.tokens, subject_attention_vector)


### Subject sentence (span, region, paragraph)

In [0]:


#ACHTUNG: this code moved to gitHub codebase, see _find_most_relevant_paragraph

attention_vector = subject_attention_vector
paragraph_attention_vector = np.zeros_like (attention_vector)
top_index=0
for i in np.nonzero(attention_vector)[0]:
  par = section.tokens_map.sentence_at_index(i)
  paragraph_len = par[1]-par[0]
  if paragraph_len:
    # TODO: Next line is weird
    # Calculate density of the matches per paragraph:
    density = attention_vector[i] / paragraph_len
    paragraph_attention_vector[par[0]: par[1]] += attention_vector[i] + density

    if paragraph_attention_vector[par[0]] > paragraph_attention_vector[top_index]:
      top_index = par[0]

par = section.tokens_map.sentence_at_index(top_index)
renderer_.render_color_text(section.tokens_cc, paragraph_attention_vector)


#### Subject extracted

In [0]:
contract.tokens_map.text_range(par)

## Value

In [0]:
for t in contract.get_tags():
  print (t)

#### hilight value

In [0]:
value_section = contract#.sections['subj'].body

value_section.calculate_distances_per_pattern(factory)
vectors = factory.make_contract_value_attention_vectors(value_section)
value_section.distances_per_pattern_dict = {**value_section.distances_per_pattern_dict, **vectors}

renderer_.render_color_text(value_section.tokens, value_section.distances_per_pattern_dict['value_attention_vector_tuned'], _range=(0,1))

In [0]:
%%time
from contract_parser import find_value_sign_currency
find_value_sign_currency(contract, ctx.pattern_factory)

In [0]:
%%time


ctx.find_contract_value_NEW(contract)

In [0]:
# from typing import List

# from contract_patterns import ContractPatternFactory
# from legal_docs import LegalDocument, extract_sum_sign_currency
# from ml_tools import SemanticTag, estimate_confidence_by_mean_top_non_zeros
# from transaction_values import complete_re

# transaction_values_re = complete_re


# def find_value_sign_currency(value_section_subdoc: LegalDocument, factory: ContractPatternFactory = None) -> List[
#   List[SemanticTag]]:
#   ''' merge dictionaries of attention vectors '''

#   if factory:
#     value_section_subdoc.calculate_distances_per_pattern(factory)
#     vectors = factory.make_contract_value_attention_vectors(value_section_subdoc)
#     value_section_subdoc.distances_per_pattern_dict = {**value_section_subdoc.distances_per_pattern_dict, **vectors}
#     attention_vector_tuned = value_section_subdoc.distances_per_pattern_dict['value_attention_vector_tuned']
#     # TODO: apply confidence to semantic tags

#   spans = [m for m in value_section_subdoc.tokens_map.finditer(transaction_values_re)]
#   values_list = [extract_sum_sign_currency(value_section_subdoc, span) for span in spans]

#   return values_list


# value_section_subdoc = contract
# value_section_subdoc.calculate_distances_per_pattern(factory)
# vectors = factory.make_contract_value_attention_vectors(value_section_subdoc)
# value_section_subdoc.distances_per_pattern_dict = {**value_section_subdoc.distances_per_pattern_dict, **vectors}
# attention_vector_tuned = value_section_subdoc.distances_per_pattern_dict['value_attention_vector_tuned']

# for value_sign_currency in find_value_sign_currency(value_section_subdoc, factory):
#   for t in value_sign_currency:
#     t.confidence *= estimate_confidence_by_mean_top_non_zeros( attention_vector_tuned[t.span[0]:t.span[1]])
#     print(t.value, t.kind, t.span, t.confidence)

#   # print(t)

# renderer_.render_color_text(value_section_subdoc.tokens, attention_vector_tuned, _range=(0, 1))

# TESTS

In [0]:
from analyser.contract_agents import *

def test_find_agent_no_comma(txt_full):
  
  _doc = LegalDocument(txt_full).parse()
  # print(_doc.text)

  tags: [SemanticTag] = find_org_names(_doc, decay_confidence=False)
  for t in tags:
    print(t)

test_find_agent_no_comma(doc.text)

In [0]:
_t = '''стороны, и ООО «Ромашка» именуемое в дальнейшем «Покупатель», в лице Петрова П. П., действующего на основании Устава, с другой стороны, совместно именуемые «Стороны», а по отдельности - «Сторона», заключили настоящий договор (далее по тексту – Договор) о нижеследующем'''
test_find_agent_no_comma(_t)

In [0]:
txt_full = 'Акционерное Общество «Газпромнефть – Терминал» именуемое в дальнейшем «Продавец», в лице генерального ' \
               'директора, действующего на основании Устава, с одной стороны, и ООО «Ромашка», именуемое в ' \
               'дальнейшем «Покупатель», в лице Петрова П.П., действующего на основании Устава, с другой стороны, совместно ' \
               'именуемые «Стороны», а по отдельности - «Сторона», заключили настоящий договор (далее по тексту – ' \
               'Договор) о нижеследующем:'
test_find_agent_no_comma(txt_full)
